Leon 1-1

Name topics
* sort tweets by highest coefficient for each topic
* look thru these tweets to determine topic name

* or have one column with the topic (being the highest coefficient)
    * can use this column to create a hue
    * bar charts ok

* no need to rearrange words

add bot_proba & determine threshold
* or, keep bot_proba & show avg bot_proba per topic!!!
    * box & whiskers

pull statistics
* x% of candidate ys tweets were bots
    * per topic
* what were bots tweeting/not tweeting about

In [5]:
import pickle

import pandas as pd
import numpy as np

import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF, LatentDirichletAllocation

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
from nltk.corpus import stopwords as nltk_stop
from custom_words.stop_words import stop_words as custom_stop

In [6]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.23.2.


In [7]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_colwidth', None)

In [8]:
data = pd.read_pickle("pickle/n2tk_limited.pick")
data.shape

(363310, 11)

In [9]:
data.columns

Index(['id', 'date', 'time', 'user_id', 'username', 'hashtags', 'trump',
       'biden', 'original', 'tweet', 'num_tokens'],
      dtype='object')

In [10]:
# with open("pickle/nmf_model.pick", 'rb') as f:
#     nmf_model = pickle.load( f)

with open("pickle/nmf_doc_topics30.pick", 'rb') as f:
    doc_topic = pickle.load( f)

with open("pickle/topic_words30.pick", 'rb') as f:
    topic_words = pickle.load( f)

In [11]:
doc_topic.shape

(363310, 30)

## Naming Topics

Before we can name our topics, we need to classify each tweet in our original DataFrame with its corresponding topic.

The values in our doc_topic matrix has rows=tweets, columns=topics, vaues=coefficients. We'll simply classify each tweet with the largest coefficient.

In [12]:
topic = doc_topic.argmax(axis=1)
data['tweet_topic'] = topic

In [13]:
# nmf_model.components_

In [14]:
# doc_topic

In [15]:
data.shape

(363310, 12)

In [16]:
doctop = pd.DataFrame(doc_topic)
doctop

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.000404,0.000130,0.000000,0.000775,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000270,0.001521,0.000162,0.000000,0.001254,0.000546,0.002367,0.000000,0.000132,0.000000,0.000841,0.000000,0.000073,0.000000,0.000261,0.000206,0.000000,0.000078,0.000107,0.000000,0.000448
1,0.000000,0.000000,0.000782,0.000142,0.000000,0.000472,0.000000,1.802203e-07,0.000343,0.000000,0.000198,0.001355,0.000423,0.000000,0.000000,0.000000,0.000319,0.000000,0.000000,0.000000,0.064040,0.001514,0.000000,0.000000,0.000011,0.000000,0.000694,0.000000,0.000515,0.000002
2,0.000000,0.000000,0.000000,0.000123,0.039796,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000021,0.000000,0.000000,0.000000,0.000000,0.000257,0.000000,0.000000,0.000000,0.000844,0.057241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000517
3,0.000425,0.000000,0.000000,0.046832,0.000000,0.000799,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000261,0.000000,0.000313,0.000000,0.000000,0.056653,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000490,0.023987,0.000000,0.000000,0.000000,0.000203,0.000489,0.000000e+00,0.000000,0.000000,0.009402,0.000000,0.032388,0.000000,0.000127,0.000000,0.000000,0.000000,0.000398,0.000000,0.000648,0.000000,0.000000,0.000000,0.000000,0.000090,0.000000,0.001582,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363305,0.001366,0.000000,0.000061,0.000000,0.002162,0.000482,0.000000,9.000112e-04,0.000000,0.000000,0.000752,0.002168,0.000000,0.035015,0.000000,0.001295,0.000000,0.000000,0.000000,0.000351,0.000000,0.000656,0.000010,0.000000,0.000000,0.002954,0.000000,0.001469,0.000000,0.000068
363306,0.000394,0.000000,0.000000,0.000297,0.000000,0.000341,0.000190,2.848399e-03,0.044850,0.000075,0.000178,0.000540,0.000000,0.000000,0.000000,0.022259,0.000000,0.000000,0.000000,0.000000,0.000500,0.001171,0.000207,0.005273,0.000000,0.000000,0.003612,0.000000,0.000195,0.013464
363307,0.000214,0.000006,0.003559,0.000281,0.000000,0.000257,0.000608,2.595221e-04,0.000000,0.000000,0.000000,0.002820,0.001422,0.000065,0.000000,0.000000,0.000000,0.000194,0.000651,0.000434,0.000000,0.040403,0.000000,0.000000,0.001162,0.001584,0.000000,0.011418,0.000424,0.001367
363308,0.001451,0.000000,0.000000,0.000463,0.000000,0.000000,0.000626,3.063218e-02,0.000000,0.000000,0.000248,0.000000,0.000599,0.000000,0.000000,0.000000,0.000000,0.000000,0.000293,0.000000,0.000000,0.000000,0.002310,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004221


Let's sort our tweets by topic coefficient, so that we can see which tweets have the highest coefficient for each topic. This will help us to identify the meaning of each topic.

In [28]:
# ensure topics start at 1 (and not zero)
if list(doctop.columns)[0] == 0:
    doctop.columns = doctop.columns + 1
    data.tweet_topic = data.tweet_topic + 1

# read topic tweets by updating topic_num
topic_num = 21
top10_ix = doctop[topic_num].sort_values(ascending=False).head(100).index # returns indexes of best tweets matching topic 1
data[['username', 'tweet', 'original', 'date']].iloc[top10_ix]


,username,tweet,original,date
731863,sherrymister3,joebiden tracker u donaldtrump v lead poll,Biden vs Trump: US presidential election 2020 poll tracker. Biden is in the lead https://t.co/jXIab3aOtG,2020-10-08
555338,barinhochimi,joebiden donaldtrump arriba national serio la loco poll,"@DonaldOfWallSt Donald hablando en serio, en la página de national polls que da todas las encuestas, en todas esta arriba y por bastante biden. Que carajo pasa?, ya estoy casi rendido loco",2020-10-26
899066,coolbreeze024,joebiden donaldtrump falcon lead tighten state swing poll,Election 2020: Biden leads Trump in polls as swing states tighten Is Biden the Atlanta Falcons? 👀 https://t.co/Wfo1nuxpic,2020-11-02
777835,gras48,joebiden point lead poll state swing donaldtrump,"@thehill New poll : PRESIDENT TRUMP LEADS BIDEN IN ALL SWING STATES BY 25 points! PRESIDENT TRUMP,MAKING AMERICA GREAT AGAIN 🇺🇸🇺🇸🇺🇸",2020-11-02
569155,timesdel,joebiden election fortnight lead poll opinion donaldtrump,"Biden Leads Trump With Fortnight to Go for Elections, But Can Opinion Polls be Trusted This Year? https://t.co/NupOqZ1un1",2020-10-18
923571,knute_scion,joebiden lead key show poll state time donaldtrump,"Biden Leads Trump in Four Key States, Poll Shows - The New York Times https://t.co/Kxw5jVQGYg via @GoogleNews",2020-11-01
729207,ronrak,joebiden say medium poll check point donaldtrump,@marthamaccallum The media says polls has Biden up over President Trump. They didn't check with my poll. It has Trump up by 20 points. #TheStory,2020-10-08
655176,alchperl,joebiden worried lead day poll take donaldtrump,Should we be worried?? Iowa Poll: Trump takes lead from Biden days before Nov. 3 election ⁦@ProjectLincoln⁩ ⁦@madrid_mike⁩ ⁦@stuartpstevens⁩ https://t.co/iU4AGfToP4,2020-10-31
11597,itselectric12,joebiden totally lead poll point hillaryclinton donaldtrump,Biden totally has this! 🤓#Trump2020MotherFckers Hillary Clinton Leads Donald Trump by 14 Points: Poll | Time https://t.co/UBJNRVVHMv,2020-10-06
933972,stella2020woof,joebiden woof r nice b poll donaldtrump,President (Pennsylvania) Biden (D) 52% Trump (R) 44% 9/29-10/27 by CCES / YouGov (B) 2703 LV Woof! Have a nice day! Poll #136092 #ElectionTwitter,2020-11-01


## Final Topic Names

After looking through the topic_words.csv file and testing each of the topics in the above cell, here's what I came up with:

In [ ]:
data['topic'] = doc_topic.argmax(axis=1)
if np.min(data.topic.to_numpy()) == 0:
    data['topic'] += 1
    
topics = {
    1:"anti-trump supporters",
    2:"#vote",
    3:"covid stats",
    4:"vp candidates",
    5:"#presidentialdebate",
    6:"trump responsible for covid",
    7:"past candidates",
    8:"#hunterbiden",
    9:"trump rally drama",
    10:"#registertovote",
    11:"#blm",
    12:"covid policy",
    13:"negative campaign ads",
    14:"voter suppression/fraud",
    15:"supreme court nom.",
    16:"#economy",
    17:"foreign affairs",
    18:"#media",
    19:"trump distrust science",
    20:"#votebiden",
    21:"#polling",
    22:"healthcare for all",
    23:"#factcheck",
    24:"town halls",
    25:"federal stimulus",
    26:"#votetrump",
    27:"trump covid diagnosis",
    28:"#covidhoax",
    29:"#breakingnews",
    30:"trump rally covid infections"
}

data['topic'] = data['topic'].map(topics)
data.to_pickle("pickle/tweet_topics.pick")

## Analyzing our Topics

Let's take a look at the number of tweets per topic.

In [23]:
(data[['tweet_topic', 'id']]
 .groupby('tweet_topic')
 .agg('count')
 .sort_values(by='id', ascending=False))


,id
tweet_topic,
1,30475
28,17061
30,16933
21,16431
8,15507
2,15116
26,14835
4,14243
17,14007


### Top 5 topics (frequency)
1. anti-trump supporters
2. covid-hoax debates
3. trump rally covid infections
4. polling
5. #hunterbiden

### Bottom 5 topics (frequency)

30. #media
29. town halls
28. voter suppression/fraud
27. register to vote
26. trump distrust science



In [ ]:
data[['username', 'date', 'topic', 'original', 'tweet']].sample(10)

In [ ]:
data.groupby('topic').agg('count')